# MCC Aggregates Forecasting - Baseline Models

This notebook implements baseline forecasting methods for MCC aggregates:
- Seasonal Naïve (multiple seasonalities)
- Random Walk
- ETS (Exponential Smoothing)


In [1]:
import pandas as pd
import numpy as np
import json
from datetime import datetime
from typing import Dict, List, Tuple
import warnings
warnings.filterwarnings('ignore')

# Suppress specific statsmodels warnings
import logging
logging.getLogger('statsmodels').setLevel(logging.ERROR)

print("Baseline Models for MCC Aggregates Forecasting")
print("=" * 50)


Baseline Models for MCC Aggregates Forecasting


In [2]:
def detect_environment():
    try:
        import google.colab
        from google.colab import drive
        drive.mount('/content/drive/')
        return 'colab', '/content/drive/MyDrive/fcst'
    except ImportError:
        return 'local', '..'

environment, base_path = detect_environment()
print(f"Environment: {environment}")
print(f"Base path: {base_path}")


Mounted at /content/drive/
Environment: colab
Base path: /content/drive/MyDrive/fcst


In [3]:
# Forecasting libraries
try:
    from statsmodels.tsa.exponential_smoothing.ets import ETSModel
    from statsmodels.tsa.holtwinters import ExponentialSmoothing
    print("✓ Statsmodels ETS imported successfully")
    ETS_AVAILABLE = True
except ImportError:
    print("⚠ Statsmodels not available - install with: pip install statsmodels")
    ETSModel = None
    ExponentialSmoothing = None
    ETS_AVAILABLE = False

# Import evaluation functions
try:
    import sys
    sys.path.append(base_path)
    from evaluation import evaluate_and_report_mcc
    print("✓ Loaded evaluation functions from project")
except ImportError:
    print("⚠ Could not import evaluation functions - will create simple evaluation")

    def evaluate_and_report_mcc(model_name, y_true, y_pred, y_train, categories, print_report=True):
        """Simple evaluation function fallback"""
        from sklearn.metrics import mean_absolute_error, mean_squared_error

        mae = mean_absolute_error(y_true, y_pred)
        rmse = np.sqrt(mean_squared_error(y_true, y_pred))

        # Simple sMAPE calculation
        smape = 100 * np.mean(2 * np.abs(y_pred - y_true) / (np.abs(y_true) + np.abs(y_pred) + 1e-8))

        # Simple RMSSE calculation
        naive_error = np.mean(np.abs(np.diff(y_train)))
        rmsse = rmse / (naive_error + 1e-8)

        metrics = {
            'MAE': mae,
            'RMSE': rmse,
            'sMAPE_w': smape,
            'RMSSE_w': rmsse
        }

        if print_report:
            print(f"\n=== {model_name} Results ===")
            print(f"MAE: {mae:.4f}")
            print(f"RMSE: {rmse:.4f}")
            print(f"sMAPE: {smape:.2f}%")
            print(f"RMSSE: {rmsse:.4f}")

        return metrics, f"{model_name} evaluation completed"


✓ Statsmodels ETS imported successfully
⚠ Could not import evaluation functions - will create simple evaluation


In [4]:
print("Loading baseline features dataset...")

# Try parquet first, fallback to CSV
baseline_parquet = f'{base_path}/data/features/baseline_statistical_full.parquet'
baseline_csv = f'{base_path}/data/features/baseline_statistical_full.csv'

try:
    df = pd.read_parquet(baseline_parquet)
    print("✓ Loaded parquet file")
except:
    try:
        df = pd.read_csv(baseline_csv)
        print("✓ Loaded CSV file")
    except:
        print("❌ Could not load baseline statistical files")
        print("Please run feature_engineering_final.py first to create the baseline features")
        exit()

# Date is already converted to datetime in the parquet file
df = df.sort_values(['client_id', 'category', 'date'])

print(f"Total rows: {len(df)}")
print(f"Unique series: {df.groupby(['client_id', 'category']).ngroups}")
print(f"Date range: {df['date'].min()} to {df['date'].max()}")
print(f"Train rows: {(df['split'] == 'train').sum()}")
print(f"Test rows: {(df['split'] == 'test').sum()}")

# Sample 1000 time series
np.random.seed(42)
series_ids = df.groupby(['client_id', 'category']).size().reset_index()
series_ids = series_ids.sample(n=min(1000, len(series_ids)), random_state=42)

# Filter data to selected series
df_sample = df.merge(series_ids[['client_id', 'category']], on=['client_id', 'category'])
print(f"Sampled series: {len(series_ids)}")
print(f"Sampled rows: {len(df_sample)}")


Loading baseline features dataset...
✓ Loaded parquet file
Total rows: 2298956
Unique series: 6026
Date range: 2009-12-28 00:00:00 to 2019-10-21 00:00:00
Train rows: 1836701
Test rows: 462255
Sampled series: 1000
Sampled rows: 382020


In [5]:
def prepare_series_data(df: pd.DataFrame) -> Dict:
    """Prepare time series data for forecasting using existing split column."""
    series_data = {}

    for (client_id, category), group in df.groupby(['client_id', 'category']):
        group = group.sort_values('date')

        # Use existing split column
        train_group = group[group['split'] == 'train']
        test_group = group[group['split'] == 'test']

        # Ensure we have enough data (at least 30 weeks of training data)
        if len(train_group) >= 30 and len(test_group) > 0:
            series_key = f"{client_id}_{category}"

            train_data = train_group['amount'].values
            test_data = test_group['amount'].values
            dates = group['date'].values

            series_data[series_key] = {
                'train': train_data,
                'test': test_data,
                'dates': dates,
                'category': category,
                'client_id': client_id,
                'split_idx': len(train_data)
            }

    return series_data

series_data = prepare_series_data(df_sample)
print(f"Series ready for forecasting: {len(series_data)}")


Series ready for forecasting: 1000


In [6]:

class SeasonalNaive:
    """Seasonal Naïve forecasting model."""

    def __init__(self, season_length: int):
        self.season_length = season_length

    def fit(self, y: np.ndarray):
        self.y_train = y
        return self

    def forecast(self, steps: int) -> np.ndarray:
        if len(self.y_train) < self.season_length:
            # If not enough data, use simple naive
            return np.full(steps, self.y_train[-1])

        # Repeat seasonal pattern
        seasonal_pattern = self.y_train[-self.season_length:]
        forecasts = []

        for i in range(steps):
            forecasts.append(seasonal_pattern[i % self.season_length])

        return np.array(forecasts)

class RandomWalk:
    """Random Walk forecasting model."""

    def fit(self, y: np.ndarray):
        self.last_value = y[-1]
        return self

    def forecast(self, steps: int) -> np.ndarray:
        return np.full(steps, self.last_value)

if ETS_AVAILABLE:
    class ETSWrapper:
        """Improved ETS (Exponential Smoothing) wrapper with better error handling."""

        def __init__(self):
            self.model = None
            self.fallback_value = None

        def fit(self, y: np.ndarray):
            self.fallback_value = y[-1]

            # Clean the data
            y_clean = np.array(y)
            y_clean = y_clean[~np.isnan(y_clean)]
            y_clean = y_clean[np.isfinite(y_clean)]

            if len(y_clean) < 10:
                return self

            # Try multiple ETS configurations in order of preference
            configs = [
                # Simple exponential smoothing
                {'error': 'add', 'trend': None, 'seasonal': None},
                # Linear trend
                {'error': 'add', 'trend': 'add', 'seasonal': None},
                # With seasonality (if enough data)
                {'error': 'add', 'trend': 'add', 'seasonal': 'add', 'seasonal_periods': min(52, len(y_clean)//3)} if len(y_clean) >= 104 else None,
                # Multiplicative error
                {'error': 'mul', 'trend': 'add', 'seasonal': None} if np.all(y_clean > 0) else None,
            ]

            # Filter out None configs
            configs = [c for c in configs if c is not None]

            for config in configs:
                try:
                    with warnings.catch_warnings():
                        warnings.simplefilter("ignore")

                        # Try new ETS implementation first
                        if 'seasonal_periods' in config:
                            self.model = ETSModel(y_clean, **config).fit(disp=False)
                        else:
                            # Use Holt-Winters for simpler models (more stable)
                            hw_config = {
                                'trend': config.get('trend'),
                                'seasonal': config.get('seasonal'),
                                'seasonal_periods': config.get('seasonal_periods', 52)
                            }
                            self.model = ExponentialSmoothing(y_clean, **hw_config).fit(optimized=True)

                        # Test if model can forecast
                        _ = self.model.forecast(1)
                        break

                except Exception:
                    continue

            return self

        def forecast(self, steps: int) -> np.ndarray:
            if self.model is None:
                return np.full(steps, self.fallback_value)

            try:
                with warnings.catch_warnings():
                    warnings.simplefilter("ignore")
                    forecast = self.model.forecast(steps)

                    # Handle any invalid forecasts
                    forecast = np.array(forecast)
                    forecast = np.where(np.isfinite(forecast), forecast, self.fallback_value)

                    return forecast

            except Exception:
                return np.full(steps, self.fallback_value)
else:
    # Simple fallback ETS if statsmodels not available
    class ETSWrapper:
        """Simple ETS fallback using exponential smoothing."""

        def __init__(self):
            self.alpha = 0.3
            self.last_value = None

        def fit(self, y: np.ndarray):
            self.last_value = y[-1]
            # Simple exponential smoothing
            smoothed = y[0]
            for val in y[1:]:
                smoothed = self.alpha * val + (1 - self.alpha) * smoothed
            self.last_value = smoothed
            return self

        def forecast(self, steps: int) -> np.ndarray:
            return np.full(steps, self.last_value)


In [7]:

def run_forecasting_experiment(series_data: Dict, models: Dict) -> Dict:
    """Run forecasting experiment on all series."""
    results = {}

    for model_name, model_class in models.items():
        print(f"\nRunning {model_name}...")

        all_y_true = []
        all_y_pred = []
        all_y_train = []
        all_categories = []

        successful_forecasts = 0
        failed_forecasts = 0

        for series_key, data in series_data.items():
            try:
                train = data['train']
                test = data['test']

                # Skip if train data is too short
                if len(train) < 10:
                    continue

                # Fit model
                if isinstance(model_class, dict):  # Seasonal naive with different seasons
                    season_length = model_class['season_length']
                    model = SeasonalNaive(season_length)
                else:
                    model = model_class()

                model.fit(train)

                # Forecast
                forecast = model.forecast(len(test))

                # Validate forecast
                if len(forecast) == len(test) and np.all(np.isfinite(forecast)):
                    # Store results
                    all_y_true.extend(test)
                    all_y_pred.extend(forecast)
                    all_y_train.extend(train)
                    all_categories.extend([data['category']] * len(test))

                    successful_forecasts += 1
                else:
                    failed_forecasts += 1

            except Exception as e:
                failed_forecasts += 1
                continue

        if successful_forecasts > 0:
            results[model_name] = {
                'y_true': np.array(all_y_true),
                'y_pred': np.array(all_y_pred),
                'y_train': np.array(all_y_train),
                'categories': np.array(all_categories),
                'successful_forecasts': successful_forecasts,
                'failed_forecasts': failed_forecasts
            }
            print(f"Successful forecasts: {successful_forecasts}")
            if failed_forecasts > 0:
                print(f"Failed forecasts: {failed_forecasts}")
        else:
            print(f"No successful forecasts for {model_name}")

    return results

# Define models
models = {
    'Random_Walk': RandomWalk,
    'ETS': ETSWrapper,
    'Seasonal_Naive_4': {'season_length': 4},
    'Seasonal_Naive_8': {'season_length': 8},
    'Seasonal_Naive_12': {'season_length': 12},
    'Seasonal_Naive_36': {'season_length': 36},
    'Seasonal_Naive_52': {'season_length': 52}
}

# Run experiments
forecast_results = run_forecasting_experiment(series_data, models)



Running Random_Walk...
Successful forecasts: 1000

Running ETS...
Successful forecasts: 1000

Running Seasonal_Naive_4...
Successful forecasts: 1000

Running Seasonal_Naive_8...
Successful forecasts: 1000

Running Seasonal_Naive_12...
Successful forecasts: 1000

Running Seasonal_Naive_36...
Successful forecasts: 1000

Running Seasonal_Naive_52...
Successful forecasts: 1000


In [8]:

def evaluate_all_models(forecast_results: Dict) -> Dict:
    """Evaluate all models and generate reports."""
    evaluation_results = {}

    for model_name, results in forecast_results.items():
        print(f"\n{'='*60}")
        print(f"EVALUATING: {model_name}")
        print(f"{'='*60}")

        # Use a representative training series for RMSSE calculation
        y_train_sample = results['y_train'][:min(len(results['y_train']), 1000)]

        overall_metrics, report = evaluate_and_report_mcc(
            model_name=model_name,
            y_true=results['y_true'],
            y_pred=results['y_pred'],
            y_train=y_train_sample,
            categories=results['categories'],
            print_report=True
        )

        evaluation_results[model_name] = {
            'overall_metrics': overall_metrics,
            'report': report,
            'successful_forecasts': results['successful_forecasts'],
            'failed_forecasts': results.get('failed_forecasts', 0)
        }

    return evaluation_results

# Run evaluation
evaluation_results = evaluate_all_models(forecast_results)



EVALUATING: Random_Walk

=== Random_Walk Results ===
MAE: 129.1289
RMSE: 203.6583
sMAPE: 81.28%
RMSSE: 2.4221

EVALUATING: ETS

=== ETS Results ===
MAE: 101.1567
RMSE: 155.8964
sMAPE: 68.41%
RMSSE: 1.8541

EVALUATING: Seasonal_Naive_4

=== Seasonal_Naive_4 Results ===
MAE: 132.6081
RMSE: 213.2287
sMAPE: 81.38%
RMSSE: 2.5360

EVALUATING: Seasonal_Naive_8

=== Seasonal_Naive_8 Results ===
MAE: 135.0889
RMSE: 228.3745
sMAPE: 81.86%
RMSSE: 2.7161

EVALUATING: Seasonal_Naive_12

=== Seasonal_Naive_12 Results ===
MAE: 135.7624
RMSE: 224.1679
sMAPE: 81.78%
RMSSE: 2.6661

EVALUATING: Seasonal_Naive_36

=== Seasonal_Naive_36 Results ===
MAE: 135.4675
RMSE: 219.8807
sMAPE: 81.68%
RMSSE: 2.6151

EVALUATING: Seasonal_Naive_52

=== Seasonal_Naive_52 Results ===
MAE: 135.1183
RMSE: 219.0510
sMAPE: 81.46%
RMSSE: 2.6052


In [9]:

def save_results_to_json(evaluation_results: Dict, filename: str = 'baseline_results.json'):
    """Save evaluation results to JSON file."""

    def convert_numpy_types(obj):
        """Convert numpy types to Python native types for JSON serialization."""
        if isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        elif isinstance(obj, dict):
            return {key: convert_numpy_types(value) for key, value in obj.items()}
        elif isinstance(obj, list):
            return [convert_numpy_types(item) for item in obj]
        else:
            return obj

    # Prepare results for JSON serialization
    json_results = {}

    # Add all models as individual entries
    json_results['models'] = {}
    for model_name, results in evaluation_results.items():
        json_results['models'][model_name] = {
            'overall_metrics': convert_numpy_types(results['overall_metrics']),
            'successful_forecasts': results['successful_forecasts'],
            'failed_forecasts': results.get('failed_forecasts', 0),
            'timestamp': datetime.now().isoformat()
        }

    # Add metadata
    json_results['metadata'] = {
        'environment': environment,
        'total_series_sampled': len(series_data),
        'evaluation_date': datetime.now().isoformat(),
        'dataset_source': 'data/features/baseline_statistical_full.parquet',
        'train_test_split': 'predefined_split',
        'models_evaluated': list(evaluation_results.keys()),
        'ets_available': ETS_AVAILABLE,
        'warning_handling': 'Improved ETS with multiple fallback configurations'
    }

    # Save to file
    output_file = f'{base_path}/MCC Aggregates Forecasting/Baseline models/{filename}'
    with open(output_file, 'w') as f:
        json.dump(json_results, f, indent=2)

    print(f"\nResults saved to: {output_file}")
    return json_results

# Save results
final_results = save_results_to_json(evaluation_results)



Results saved to: /content/drive/MyDrive/fcst/MCC Aggregates Forecasting/Baseline models/baseline_results.json


In [10]:
print(f"\n{'='*80}")
print("BASELINE MODELS EXPERIMENT SUMMARY")
print(f"{'='*80}")
print(f"Environment: {environment}")
print(f"Total series processed: {len(series_data)}")
print(f"Models evaluated: {len(evaluation_results)}")
print(f"ETS available: {ETS_AVAILABLE}")

print("\n" + "="*50)
print("ALL MODELS PERFORMANCE (by sMAPE_w):")
print("="*50)
all_scores = {}
for model_name, results in evaluation_results.items():
    if 'sMAPE_w' in results['overall_metrics']:
        all_scores[model_name] = results['overall_metrics']['sMAPE_w']
        print(f"{model_name}: {results['overall_metrics']['sMAPE_w']:.4f}")

print("\n" + "="*50)
print("OVERALL RANKING (by sMAPE_w):")
print("="*50)
if all_scores:
    sorted_models = sorted(all_scores.items(), key=lambda x: x[1])
    for i, (model, score) in enumerate(sorted_models):
        print(f"{i+1}. {model}: {score:.4f}")

print(f"\nResults saved to: baseline_results.json")
print("Experiment completed successfully!")


BASELINE MODELS EXPERIMENT SUMMARY
Environment: colab
Total series processed: 1000
Models evaluated: 7
ETS available: True

ALL MODELS PERFORMANCE (by sMAPE_w):
Random_Walk: 81.2765
ETS: 68.4099
Seasonal_Naive_4: 81.3838
Seasonal_Naive_8: 81.8587
Seasonal_Naive_12: 81.7820
Seasonal_Naive_36: 81.6750
Seasonal_Naive_52: 81.4566

OVERALL RANKING (by sMAPE_w):
1. ETS: 68.4099
2. Random_Walk: 81.2765
3. Seasonal_Naive_4: 81.3838
4. Seasonal_Naive_52: 81.4566
5. Seasonal_Naive_36: 81.6750
6. Seasonal_Naive_12: 81.7820
7. Seasonal_Naive_8: 81.8587

Results saved to: baseline_results.json
Experiment completed successfully!
